# BeerVolume Prediction Demo - Training Pipeline (2)

This is the seocnd part of a short demonstration of how you can use Hopsworks for creating a Machine Learning System that creates predictions. The hypothetical use case is that of a **bar owner** who would want to predict how much beer will be consumed in his bar, based on past trends and behaviour.

![](https://lh3.googleusercontent.com/blogger_img_proxy/ALY8t1uqu0YUTdfoFJYGV2r9a2iHEewpP3daVa9J3qzCzV3rZm8EX8YyhHhOHbfG450AhHYQXu6Hgf8pj2fTpSzg4uio4X_qv9TTEfMnEtO6rYLevnGBxF6sO97tGeYyzaAkGSyVBnw8WtWS1P_2RLY=s0-d)

In this second part of the Machine Learning system that we are creating, we are going to focus on the **Training Pipeline** required to make the predictions that we want.

## <span style="color:#ff5f27">📝 Code Library Imports and installations</span>

### Installing Supporting libraries
We also need to install the Kaleido libraries so that we can use these during our notebook exploration:

In [1]:
!pip install -U kaleido # For Plotly Image export
!pip install sklearn

### Other imports
We are importing a number of other libaries, among which several classes from  `beervolume.py` and `averages.py`, into this notebook, as we will use it later on.

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import os
import joblib
from beervolume import plot_prediction_test
from functions import predict_id
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>
As we have already installed Hopsworks on this system, all we now need to do is import the library into this notebook, and start establishing the connection to the Hopsworks feature store.

In [3]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) BeerVolumePrediction
	 (2) rixdemo

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/294039
Connected. Call `.close()` to terminate connection gracefully.


Since we have already created the necessary feature groups in our previous notebook, we can now just import these into our training pipeline. Remember we had two feature groups:
1. a `beervolume` feature group that contained the synthetic data that we had created,
2. a `beervolume_averages` feature group that contained a number of _engineered_ features that we derived from the `beervolume` feature group in order to have better predictions later on.

We retrieve bother feature groups:

In [4]:
# Retrieve feature groups
beervolume_fg = fs.get_feature_group(
    name='beervolume',
    version=1,
)
beervolume_averages_fg = fs.get_feature_group(
    name='beervolume_averages',
    version=1,
)

## <span style="color:#ff5f27">🔪 Preparing our Training pipeline: Feature Selection </span>
Using the Hopsworks feature store, we can very easily select the specific subset that we need for our training pipeline, and be confident that we will be using the correct training dataset in our Machine Learning process:

In [5]:
# Select features for training dataset
query = beervolume_fg.select_all() \
    .join(beervolume_averages_fg.select_except(['date']))
query.show(5)

Finished: Reading data from Hopsworks, using ArrowFlight (7.23s) 


,date,id,beervolume,ma_7,ma_14,ma_30,daily_rate_of_change,volatility_30_day,ema_02,ema_05,rsi
0,2023-09-01,0,200.0,0.0,0.0,0.0,0.000000,0.0,200.000000,200.000000,0.0
1,2023-09-02,0,203.6,0.0,0.0,0.0,1.800000,0.0,202.000000,202.400000,0.0
2,2023-09-03,0,202.0,0.0,0.0,0.0,-0.785855,0.0,202.000000,202.171429,0.0
3,2023-09-05,0,202.4,0.0,0.0,0.0,0.198020,0.0,202.135501,202.293333,0.0
4,2023-09-06,0,205.0,0.0,0.0,0.0,1.284585,0.0,202.987625,203.690323,0.0


## <span style="color:#ff5f27">🤖 Preparing our Training pipeline: Transformation Functions </span>
Using the Hopsworks feature store, we can easily apply a set of transformation functions that will apply to our training pipeline:

In [6]:
# Load transformation function
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")

# Define a list of feature names
feature_names = [
    'ma_7', 'ma_14', 'ma_30', 'daily_rate_of_change', 'volatility_30_day', 'ema_02', 'ema_05', 'rsi'
]

# Map features to transformations
transformation_functions = {
    feature_name: min_max_scaler
    for feature_name in feature_names
}
transformation_functions

{'ma_7': <hsfs.transformation_function.TransformationFunction at 0x15b275390>,
 'ma_14': <hsfs.transformation_function.TransformationFunction at 0x15b275390>,
 'ma_30': <hsfs.transformation_function.TransformationFunction at 0x15b275390>,
 'daily_rate_of_change': <hsfs.transformation_function.TransformationFunction at 0x15b275390>,
 'volatility_30_day': <hsfs.transformation_function.TransformationFunction at 0x15b275390>,
 'ema_02': <hsfs.transformation_function.TransformationFunction at 0x15b275390>,
 'ema_05': <hsfs.transformation_function.TransformationFunction at 0x15b275390>,
 'rsi': <hsfs.transformation_function.TransformationFunction at 0x15b275390>}

## <span style="color:#ff5f27">⚙️ Feature View Creation leveraging feature groups, feature selection and feature transformations </span>
Using the feature groups, the selection that we made above and finally the associated transformation functions, we can create a **feature view** in the Hopsworks feature store:

In [7]:
# Get or create the 'beervolume_fv' feature view
feature_view = fs.get_or_create_feature_view(
    name='beervolume_fv',
    version=1,
    query=query,
    labels=["beervolume"],
    transformation_functions=transformation_functions,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/294039/fs/293958/fv/beervolume_fv/version/1


# <span style="color:#ff5f27">🏋️ Training Dataset Creation based on the Feature View</span>
The next part of our process will focus on the creation of a Training dataset based on the feature view that we have prepared above.

In [8]:
# Get training and testing sets
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    description='Beer Volume Dataset',  # Provide a description for the dataset split
    train_start='2023-09-01',      # Start date for the training set
    train_end='2023-11-01',        # End date for the training set
    test_start='2023-11-01',       # Start date for the testing set
    test_end=datetime.today().strftime("%Y-%m-%d"),  # End date for the testing set (current date)
)

Finished: Reading data from Hopsworks, using ArrowFlight (7.32s) 


In [9]:
X_train.head(3)

,date,id,ma_7,ma_14,ma_30,daily_rate_of_change,volatility_30_day,ema_02,ema_05,rsi
0,2023-09-01,0,0.0,0.0,0.0,0.465254,0.0,0.216471,0.186116,0.336139
1,2023-09-02,0,0.0,0.0,0.0,0.532809,0.0,0.271274,0.242657,0.336139
2,2023-09-03,0,0.0,0.0,0.0,0.435761,0.0,0.271274,0.237273,0.336139


In [10]:
y_train.head(3)

,beervolume
0,200.0
1,203.6
2,202.0


In [11]:
# Sort the training features by the 'date' column
X_train = X_train.sort_values("date")

# Reindex the target 'y_train' to match the sorted order of 'X_train'
y_train = y_train.reindex(X_train.index)

# Sort the testing features by the 'date' column
X_test = X_test.sort_values("date")

# Reindex the target 'y_test' to match the sorted order of 'X_test'
y_test = y_test.reindex(X_test.index)

# Extract and store the 'date' column as a separate DataFrame for both training and testing sets
train_date = pd.DataFrame(X_train.pop("date"))
test_date = pd.DataFrame(X_test.pop("date"))

# <span style="color:#ff5f27">🧬 Using the training dataset to create a Machine Learning Model</span>

In order to now create a predictive model out of our data, we will use the **XGBoost Regressor** to examine the training dataset and figure out a predictive model. XGBoost regressor is a powerful and highly effective machine learning algorithm for regression problems. XGBoost is known for its ability to handle complex relationships in the data, handle missing values, and provide accurate predictions. It's a popular choice in the data science community due to its robustness and excellent predictive performance, making it well-suited for our specific problem.

In [12]:
# Initialize the XGBoost regressor
model = xgb.XGBRegressor()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the validation set
y_test_pred = model.predict(X_test)

# Calculate RMSE on the validation set
mse = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 2.1531443037153233


In [13]:
# Make predictions for a specific ID (ID=1) using the 'predict_id' function
prediction_for_id = predict_id(
    1, 
    X_test, 
    model,
)

# Generate a Plotly figure for visualizing the predictions
fig = plot_prediction_test(
    1, 
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    train_date, 
    test_date, 
    prediction_for_id,
)

# Display the generated Plotly figure
fig.show()

## <span style="color:#ff5f27">⚙️ Creating the Model Schema </span>
Now that we know what the input training features and the target/output variables will look like, we can create a schema for this and save this to a dictionary.

In [14]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Create an input schema using the training features
input_schema = Schema(X_train.values)

# Create an output schema using the target variable
output_schema = Schema(y_train)

# Create a model schema using the input and output schemas
model_schema = ModelSchema(
    input_schema=input_schema, 
    output_schema=output_schema,
)

# Convert the model schema to a dictionary
model_schema.to_dict()

{'input_schema': {'tensor_schema': {'shape': '(192789, 9)',
   'type': 'float64'}},
 'output_schema': {'columnar_schema': [{'name': 'beervolume',
    'type': 'float64'}]}}

## <span style="color:#ff5f27">📝 Registering the model in the Hopsworks registry</span>
We will also save and persist the model in Hopsworks, in case future decisions would need to be retraced to a specific version of the prediction system:

In [15]:
# Specify the directory for saving the model
model_dir = "beervolume_model"

# Check if the directory exists, and create it if not
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

# Save the trained XGBoost model using joblib
joblib.dump(model, f'{model_dir}/xgboost_beervolume_model.pkl')

# Write the generated Plotly figure image to the specified directory
fig.write_image(f'{model_dir}/model_prediction.png')

/Users/rvanbruggen/.pyenv/versions/3.11.6/lib/python3.11/site-packages/kaleido/scopes/base.py:188: DeprecationWarning:

setDaemon() is deprecated, set the daemon attribute instead



In [17]:
# Get the model registry from the project
mr = project.get_model_registry()

# Create a Python model in the model registry named 'xgboost_price_model'
beervolume_model = mr.python.create_model(
    name="xgboost_beervolume_model", 
    metrics={"MSE": mse},           # Specify metrics (Mean Squared Error)
    model_schema=model_schema,      # Provide the model schema
    input_example=X_train.sample(), # Provide an example of the input data
    description="Beervolume Predictor",  # Add a description for the model
)

# Save the model to the specified directory
beervolume_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/294039/models/xgboost_beervolume_model/1


Model(name: 'xgboost_beervolume_model', version: 1)

# <span style="color:#ff5f27">🚀 Beervolume Consumption Prediction Model Deployment</span>

**About Model Serving**

Models can be served via KFServing or "default" serving, which means a Docker container exposing a Flask server. For KFServing models, or models written in Tensorflow, we do not need to write a prediction file. However, for sklearn models using default serving, you do need to proceed to write a prediction file.

We will not go into this at this point - and wrap up the 2nd part of our prediction system with this training example.

----

## <span style="color:#ff5f27">📎 Predictor script for Python models</span>

Scikit-learn and XGBoost models are deployed as Python models, in which case you need to provide a Predict class that implements the predict method. The `predict()` method invokes the model on the inputs and returns the prediction as a list.

The `init()` method is run when the predictor is loaded into memory, loading the model from the local directory it is materialized to, ARTIFACT_FILES_PATH.

The directive **"%%writefile"** writes out the cell before to the given Python file. We will use the **predict_example.py** file to create a deployment for our model.

In [18]:
%%writefile predict_example.py
import os
import numpy as np
import pandas as pd
import hsfs
import joblib


class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        # get feature store handle
        fs_conn = hsfs.connection()
        self.fs = fs_conn.get_feature_store()
        
        # get feature view
        self.fv = self.fs.get_feature_view("beervolume_fv", 1)
        
        # initialize serving
        self.fv.init_serving(1)

        # load the trained model
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/xgboost_beervolume_model.pkl")
        print("Initialization of Beervolume model Complete")

    
    def predict(self, id_value):
        """ Serves a Beervolume prediction request usign a trained model"""
        # Retrieve feature vectors
        feature_vector = self.fv.get_feature_vector(
            entry = {'id': id_value[0]}
        )
        return self.model.predict(np.asarray(feature_vector[1:]).reshape(1, -1)).tolist()

Writing predict_example.py


This script needs to be put into a known location in the Hopsworks file system. Let's call the file predict_example.py and put it in the Models directory.

In [19]:
# Get the dataset API from the project
dataset_api = project.get_dataset_api()

# Upload the file "predict_example.py" to the "Models" dataset, overwriting if it already exists
uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)

# Create the full path to the uploaded predictor script
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 0.000%|          | 0/1027 elapsed<00:00 remaining<?

---

## <span style="color:#ff5f27">🚀 Create the deployment</span>

Here, you fetch the model you want from the model registry and define a configuration for the deployment. For the configuration, you need to specify the serving type (default or KFserving).

In [20]:
# Deploy the 'beervolume_model'
deployment = beervolume_model.deploy(
    name="beervolumeonlinemodeldeployment",  # Specify the deployment name
    script_file=predictor_script_path,  # Provide the path to the predictor script
)

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/294039/deployments/157698
Before making predictions, start the deployment by using `.start()`


In [21]:
# Start the deployment and wait for it up to 180 seconds
deployment.start(await_running=180)

  0%|          | 0/5 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [22]:
# Get the current state of the deployment and describe it
deployment_state = deployment.get_state().describe()

{
    "available_instances": 0,
    "available_transformer_instances": 0,
    "condition": {
        "reason": "Deployment is ready, but idle. Higher latencies are expected in the first predictions",
        "status": true,
        "type": "READY"
    },
    "deployed": "2023-12-12T11:14:35.000Z",
    "hopsworks_inference_path": "/project/294039/inference/models/beervolumeonlinemodeldeployment",
    "model_server_inference_path": "/v1/models/beervolumeonlinemodeldeployment",
    "revision": "82290516",
    "status": "Idle"
}


In [25]:
# Predict beervolume for the 1 ID
deployment.predict({'instances': [1]})

RestAPIError: Metadata operation error: (url: http://acfdd5a4a839249e8bb85d8b9651a20b-928877002.us-east-2.elb.amazonaws.com/v1/models/beervolumeonlinemodeldeployment:predict). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"error":"HTTPError : HTTP 500: \'XGBModel\' object has no attribute \'gpu_id\'"}', error code: , error msg: , user msg: 

 Check the model server logs by using `.get_logs()`

---